In [1]:
pip install requests beautifulsoup4 pandas selenium pillow

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/1e/db/4254e3eabe8020b458f1a747140d32277ec7a271daf1d235b70dc0b4e6e3/requests-2.32.5-py3-none-any.whl.metadata
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Obtaining dependency information for beautifulsoup4 from https://files.pythonhosted.org/packages/94/fe/3aed5d0be4d404d12d36ab97e2f1791424d9ca39c2f754a6285d59a3b01d/beautifulsoup4-4.14.2-py3-none-any.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/8e/59/712db1d7040520de7a4965df15b774348980e6df45c129b8c64d0dbe74ef/pandas-2.3.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/74/9e/642a355e43a4ebf68bc4f00dd4ab264f635079c5dc7ed6d9991a0c2be3d7/selenium-4.36.0-py3-none-any.whl.metadata
  Obtaining dependency information for pillow from https://files.pythonhosted.org/packages/4d/42/aaca386de5cc8


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1) 설정&공통 유틸

In [25]:
# %% [markdown]
# # 인사아트센터 현재 전시 크롤러 (텍스트 + 이미지)

# %%
import os
import re   # 정규식
import time
import json
from urllib.parse import urljoin, urlparse

import requests # HTTP 요청
from bs4 import BeautifulSoup   # HTML 파싱
import pandas as pd # 표/CSV저장

BASE = "https://www.insaartcenter.com"
LIST_URL = "https://www.insaartcenter.com/bbs/board.php?bo_table=exhibition_current"

# 브라우저처럼 보이는 UA (이미지 핫링크 방지 대응)
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/127.0.0.0 Safari/537.36"
    )
}

# 옵션들
SAVE_IMG = True                 # 이미지를 파일로 저장할지
IMG_DIR = "insaart_images"      # 이미지 저장 폴더
MAX_IMGS_PER_EXHIBIT = 5        # 전시별 이미지 최대 저장 수(None이면 제한 없음)
POLITE_DELAY = 1.0              # 상세 페이지 간 지연(초)
TIMEOUT = 20                    # 요청 타임아웃(초)

FIELD_LABELS = ["전시명", "부주제", "전시장소", "전시기간", "작가", "전시관"]


# 2) 목록 -> 상세 링크 수집

In [58]:
# %%
def get_detail_links():
    r = requests.get(LIST_URL, headers=HEADERS, timeout=TIMEOUT)    # 목록 페이지를 GET
    r.raise_for_status()    # 상태 코드 4xx/5xx일 경우 raise_for_status()가 즉시 예외를 던져서 이후 로직 중단
    soup = BeautifulSoup(r.text, "html.parser")  # lxml미설치 이슈 방지
    anchors = soup.select('a[href*="bo_table=exhibition_current"][href*="wr_id="]')
    # 절대 URL변환 & 중복 제거
    links, seen = [], set()
    for a in anchors:
        href = a.get("href")
        if not href:
            continue
        url = urljoin(BASE, href)   # 상대경로 -> 절대경로
        if "wr_id=" in url and url not in seen:
            seen.add(url)   # 중복 방지
            links.append(url)
    return links
# 수집된 상세 링크 총 수, 앞에 5개만 확인
detail_links = get_detail_links()
len(detail_links), detail_links[:5]


(8,
 ['https://www.insaartcenter.com/bbs/board.php?bo_table=exhibition_current&wr_id=251',
  'https://www.insaartcenter.com/bbs/board.php?bo_table=exhibition_current&wr_id=260',
  'https://www.insaartcenter.com/bbs/board.php?bo_table=exhibition_current&wr_id=261',
  'https://www.insaartcenter.com/bbs/board.php?bo_table=exhibition_current&wr_id=262',
  'https://www.insaartcenter.com/bbs/board.php?bo_table=exhibition_current&wr_id=263'])

# 3) 상세 페이지 파싱 함수

In [69]:
# 줄바꿈/공백 정리용 헬퍼
def normalize_desc(text: str) -> str:
    if not text:
        return text
    # CR 제거
    text = text.replace("\r", "")
    # 연속 공백을 1칸으로
    text = re.sub(r"[ \t\u00A0]+", " ", text)
    # 3개 이상 줄바꿈 -> 2개(단락 유지)
    text = re.sub(r"\n{3,}", "\n\n", text)
    # 단일 줄바꿈(단락 아님)은 공백으로 치환
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)
    # 문장부호 앞의 공백 제거 (.,;:!? ) 등
    text = re.sub(r"\s+([,.;:!?])", r"\1", text)
    # 여는 괄호/따옴표 뒤 공백 제거
    text = re.sub(r"([\(\[\{‘“])\s+", r"\1", text)
    # 닫는 괄호/따옴표 앞 공백 제거
    text = re.sub(r"\s+([\)\]\}’”])", r"\1", text)
    # 앞뒤 공백 정리
    return text.strip()


In [81]:
# %% 텍스트 필드 추출 전용
def extract_text_fields(detail_url):
    r = requests.get(detail_url, headers=HEADERS, timeout=TIMEOUT)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

	# (id="bo_v")을 우선적으로 찾고, 없으면 페이지 전체(soup)을 대상으로 찾음
    outer = soup.select_one("#bo_v") or soup
    # HTML 태그를 제거하고 순수 텍스트만 추출, replace("\u200b", "")는 눈에 보이지 않는 특수 문자 제거
    text_all = outer.get_text("\n", strip=True).replace("\u200b", "")

    # 웹페이지에서 찾고 싶은 정보의 종류를 정의한 리스트
    wanted = {
        "전시명": "전시명",
        "부주제": "부주제",
        "전시장소": "전시장소",
        "전시기간": "전시기간",
        "작가": "작가",
        "전시관": "전시관",
    }
    # wanted 목록을 기반으로, 결과를 담을 비어있는 딕셔너리 생성. {'전시명': ' ', '작가': ' '}형태
    data = {v: "" for v in wanted.values()}

    # HTML을 보니 정보가 <tr>, <th>, <td>에 들어가 있는 것을 확인 했으니 이렇게 <table>형식의 정보를 가져옴
    tables = outer.select("table")
    for table in tables:
        for tr in table.select("tr"):
            th = tr.find("th")  # 키값
            td = tr.find("td")  # value
            if not th or not td:
                continue
            key = th.get_text(separator=" ", strip=True).replace("\u200b", "")
            val = td.get_text(separator="\n", strip=True)
            key = key.replace(":", "").strip()
            # 찾은 키가 원하던 목록(wanted)에 있는지 확인, not data[wanted[key]]에선 해당 정보가 아직 비어있는 경우에만 값을 채워넣고
            # 다른 정보가 있을 경우 덮어쓰지 않음
            if key in wanted and not data[wanted[key]]:
                data[wanted[key]] = val

    # 만약 table형식으로 탐색을 못 했을 경우 실행. data딕셔너리 값들이 모두 비어있는지 확인
    if not any(data.values()):
        for label in wanted.keys():
            # 페이지 전체 텍스트를 대상으로 정규식을 사용해 원한느 정보를 찾음
            # rf"{label}\s*:?\s*([^\n\r]+)"는 '작가'라는 글자 뒤에 콜론이 있거나 없고, 약간의 공백 뒤에 오는, 줄바꿈이 아닌 모든 글자를 찾아낸다.
            # 즉, 부수적인 요소를 빼고 텍스트만 추출
            m = re.search(rf"{label}\s*:?\s*([^\n\r]+)", text_all)  
            if m:
                data[wanted[label]] = m.group(1).strip()

    desc = ""
    # "전시회 설명"이라는 텍스트를 기준으로 전체 텍스트를 두 조각으로 나눔
    chunks = re.split(r"전시회\s*설명", text_all, maxsplit=1)
    if len(chunks) == 2:
        # chunks[1]나눈 두 조각 중 뒷부분, 즉 전시회 설명에 해당하는 텍스트 덩어리를 선택
        # r"(이전글|다음글|개인정보처리방침|이메일무단수집거부)" 설명 덩어리에서 불 필요한 부분 모두 잘라냄
        desc = re.split(r"(이전글|다음글|개인정보처리방침|이메일무단수집거부)", chunks[1])[0].strip()
    data["전시회 설명"] = normalize_desc(desc)  # 줄바꿈 심한 것 함수로 처리

    data["url"] = detail_url
    return data


In [92]:
# %% 인사아트센터 이미지 수집/저장 유틸 (정리본)

import os, re, requests
from urllib.parse import urljoin, urlparse, parse_qs, quote

# ==============================================================
# 1) 이미지 URL 수집: view_image.php 링크만 수집
# ==============================================================

def collect_image_urls_prefer_original(detail_url: str) -> list[str]:
    """
    문서 전체에서 a[href*="view_image.php"] 만 수집하여 반환.
    (썸네일/원본 추정은 여기서 하지 않음)
    """
    # 일회성으로 요청을 보내느 대신 Session객체를 사용. 여러 요청을 '하나의 브라우저 세션처럼' 묶어서 효율, 안정성을 올리기 위함
    with requests.Session() as s:
        s.headers.update(HEADERS)
        r = s.get(detail_url, timeout=TIMEOUT)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

    urls, seen = [], set()
    # 그냥 보이는 이미지<img>를 찾는게 아니라, 클릭했을 때 이미지를 보여주는 전용 페이지로 연결되는 링크를 찾음
    # 이는 해당 웹사이트가 전시 작품 이미지를 이런 방식으로 제공한다는 것을 파악하고, 핵심 링크만 골라내는 전략임
    for a in soup.select('a[href*="view_image.php"]'):
        href = a.get("href")
        if not href:
            continue
        u = urljoin(detail_url, href)
        if "no_profile.gif" in u:
            continue
        if u not in seen:
            seen.add(u)
            urls.append(u)
    return urls

# ==============================================================
# 2) 파일명/경로 유틸
# ==============================================================

# 웹사이트의 구조를 분석하여 원본 이미지 URL과 썸네일 이미지 URL이 어떤 패턴을 가지는지 파악했기 때문에 이 함수들은
# 파일명(fn)을 기반으로 그 패턴에 맞는 URL을 조립하여 만들어 낸다.
def _direct_file_url_from_fn(fn: str) -> str:
    """원본 파일 추정 경로: /data/file/exhibition_current/<fn>"""
    return "https://www.insaartcenter.com/data/file/exhibition_current/" + quote(fn)

def _thumb_url_from_fn(fn: str, size="600x0") -> str:
    """썸네일 파일 추정 경로: /data/file/exhibition_current/thumb-<stem>_<size>.<ext>"""
    m = re.match(r"(.+)\.([A-Za-z0-9]+)$", fn)
    base = "https://www.insaartcenter.com/data/file/exhibition_current/"
    if not m:
        return base + quote(fn)
    stem, ext = m.groups()
    return base + quote(f"thumb-{stem}_{size}.{ext}")

def _filename_from_url_or_headers(url: str, resp) -> str:
    """다운로드 응답/URL에서 저장 파일명을 안전하게 결정"""
    # view_image.php의 fn 파라미터 우선 -> 파일명을 기준으로 저장
    if "view_image.php" in url:
        qs = parse_qs(urlparse(url).query)
        fn = qs.get("fn", [None])[0]
        if fn:
            return fn

    # 서버 응답 헤더의 Content-Disposition값을 확인해서 저장. 가끔 서버가 직접 어떤 파일로 저장하라고 알려주기도 함.
    cd = resp.headers.get("Content-Disposition", "")
    m = re.search(r'filename\*?=(?:UTF-8\'\')?"?([^";]+)"?', cd)
    if m:
        return m.group(1)

    # URL basename -> URL 경로의 마지막 부분을 파일명으로 사용
    base = os.path.basename(urlparse(url).path)
    if base:
        return base

    # 최후: Content-Type 기반 확장자 -> Content-Type 헤더를 보고 파일 확장자 추측
    ctype = (resp.headers.get("Content-Type") or "").lower()
    if "png" in ctype: return "image.png"
    if "webp" in ctype: return "image.webp"
    return "image.jpg"

# ==============================================================
# 3) 중복 방지 헬퍼 (핵심: 같은 이미지 2번 저장되는 문제 해결)
# ==============================================================

# 하나의 이미지라도 여러개의 다른 URL을 가질 수 있다.
# 예) 이미지 뷰어링크, 원본 이미지 파일 직접 링크, 썸네일 이미지 링크
# 단순히 URL만으로 중복을 검사하면, 이 3가지 이미지를 서로 다른걸로 착각하여 3번 다운받게 됨
def _norm_name_from_url(u: str) -> str: # URL에서 파일명을 추출한 뒤, 썸네일을 의미하는 thumb-접두사나 _600x0같은 크기 정보를 제거하여 파일명 '정규화'
    """
    URL 파일명을 썸네일 변형 없이 표준화해서 키로 사용
    - thumb- 접두 제거
    - _<w>x<h> 접미 제거 (예: _600x0)
    """
    name = os.path.basename(urlparse(u).path)  # 예: thumb-abc_600x0.jpg
    name = re.sub(r'^thumb-', '', name, flags=re.IGNORECASE)
    name = re.sub(r'_(\d+)x(\d+)(?=\.[A-Za-z0-9]+$)', '', name)
    return name.lower()

# URL의 형태와 상관없이 각 이미지를 식별할 수 있는 고유한 Key를 생성
def dedupe_img_urls_by_key(img_urls: list[str]) -> list[str]:
    """
    - view_image.php면 fn 파라미터를 키로
    - 그 외는 표준화 파일명을 키로
    키 기준으로 중복 제거하여 같은 이미지를 한 번만 저장
    """
    uniq, seen = [], set()
    for u in img_urls:
        if "view_image.php" in u:
            fn = parse_qs(urlparse(u).query).get("fn", [None])[0]
            key = fn.lower() if fn else _norm_name_from_url(u)
        else:
            key = _norm_name_from_url(u)
        # 정규화된 URL이 전에 생성된 적이 없을 경우 최종 다운로드 목록인 uniq에 저장
        if key not in seen:
            seen.add(key)
            uniq.append(u)
    return uniq

# ==============================================================
# 4) 이미지 다운로드 (세션 프리히트 + Referer + direct/썸네일 폴백)
# ==============================================================

def download_images_from_urls(detail_url: str, img_urls: list[str],
                              img_dir=IMG_DIR, max_imgs: int | None = MAX_IMGS_PER_EXHIBIT) -> list[str]:
    """
    입력된 img_urls(보통 view_image.php)로부터 이미지를 저장.
    - 먼저 상세 페이지를 열어 쿠키/세션 확보
    - 각 URL은 view_image → direct(data/file/...) → thumbnail 순으로 시도
    - 같은 이미지는 dedupe로 한 번만 저장
    """
    if not img_urls:
        return []

    # ✅ 같은 이미지(같은 fn/파일명) 사전 중복 제거
    img_urls = dedupe_img_urls_by_key(img_urls)

    s = requests.Session()
    s.headers.update({
        **HEADERS,
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    })

    # 세션/쿠키 프리히트 (매우 중요)
    # 먼저 전시회 상세 페이지를 Session객체로 방문. 이 과정을 통해 필요한 쿠키를 얻어, 이어지는 이미지 요청들이 마치 정상적인 사용자가
    # 사이트를 둘러보다 이미지를 클릭하는 것 처럼 보이게 만듦.
    s.get(detail_url, timeout=TIMEOUT)

    # 전시글별 하위 폴더
    wr_id = detail_url.split("wr_id=")[-1]
    subdir = os.path.join(img_dir, re.sub(r"[^0-9A-Za-z_-]", "_", wr_id))
    os.makedirs(subdir, exist_ok=True)

    saved = []
    tried = 0

    for u in img_urls:
        if max_imgs is not None and tried >= max_imgs:
            break
        tried += 1

        # 하나의 이미지를 다운로드할 때, 무작정 하나의 URL만 시도하는게 아니라 성공확률과 이미지 품질을 고려하여 후보 목록을 생성
        # 후보: 1) view_image.php  2) direct  3) thumbnail
        candidates = [u]
        fn = None
        # view_image.php -> 뷰어 페이지
        if "view_image.php" in u:
            qs = parse_qs(urlparse(u).query)
            fn = qs.get("fn", [None])[0]
            if fn:
                candidates.append(_direct_file_url_from_fn(fn))     # 추측한 이미지 링크
                candidates.append(_thumb_url_from_fn(fn, "600x0"))  # 추측한 썸네일 링크

        ok = False
        for cu in candidates:
            try:
                r = s.get(
                    cu,
                    headers={
                        **HEADERS,
                        "Referer": detail_url,  # 🔑 핫링크 방지 회피 -> 서버의 접근 차단을 피하기 위한 가장 결정적인 부분
                        "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
                    },
                    timeout=TIMEOUT,
                    allow_redirects=True,
                )
                ctype = (r.headers.get("Content-Type") or "").lower()
                # 다운르도가 성공하고, 내용물이 진짜 이미지일 경우에만 파일을 저장.
                if r.status_code == 200 and r.content and "image" in ctype:
                    name = _filename_from_url_or_headers(cu, r)
                    if "." not in os.path.basename(name):
                        # 확장자 보정
                        if "png" in ctype: name += ".png"
                        elif "webp" in ctype: name += ".webp"
                        else: name += ".jpg"

                    base, ext = os.path.splitext(name)
                    final = os.path.join(subdir, name)
                    k = 1
                    # 만약 같은 이름의 파일이 이미 존재할 경우 접미사를 붙여서 덮어쓰기를 방지
                    while os.path.exists(final):
                        final = os.path.join(subdir, f"{base}_{k}{ext}")
                        k += 1

                    with open(final, "wb") as f:
                        f.write(r.content)
                    saved.append(final)
                    ok = True
                    break
                else:
                    print(f"[이미지 응답 이상] {r.status_code} {cu} (ctype={ctype})")
            except Exception as e:
                print(f"[이미지 실패] {cu} -> {e}")

        if not ok and fn:
            print(f"[스킵] 다운로드 실패: fn={fn}")

    return saved


In [93]:
# 텍스트 + 이미지 통합 (이미지 저장 포함)
def extract_all(detail_url, save_img=SAVE_IMG, max_imgs=MAX_IMGS_PER_EXHIBIT):
    # 1) 텍스트 필드
    fields = extract_text_fields(detail_url)

    # 2) 이미지 URL 수집 (view_image.php만 수집하는 함수 버전 권장)
    img_urls = collect_image_urls_prefer_original(detail_url)  # <- 함수명 맞춰 쓰세요

    fields["전시 사진"] = img_urls

    # 3) 실제 이미지 저장
    if save_img and img_urls:
        # 이 함수는 반드시 "세션 프리히트 + Referer + direct/썸네일 폴백" 버전이어야 합니다.
        saved = download_images_from_urls(detail_url, img_urls, img_dir=IMG_DIR, max_imgs=max_imgs)
    else:
        saved = []

    fields["저장된_이미지_경로"] = saved
    fields["url"] = detail_url
    return fields


# 4) 전체 실행 + DataFrame/CSV 저장

In [94]:
# 실행
def crawl_current_exhibitions():
    links = get_detail_links()
    results = []
    for link in links:
        try:
            item = extract_all(link, save_img=SAVE_IMG, max_imgs=MAX_IMGS_PER_EXHIBIT)
            results.append(item)
            time.sleep(POLITE_DELAY)
        except Exception as e:
            results.append({"url": link, "error": str(e)})
    return results

results = crawl_current_exhibitions()
df = pd.DataFrame(results)
len(df), df.columns.tolist()

[이미지 응답 이상] 200 https://www.insaartcenter.com/bbs/view_image.php?bo_table=exhibition_current&fn=f908992e10ec9c87f45e0f975130eaa9_RwLldWx4_d4c7e6dc0a620dc6b59321d7549ddc6f221d8008.jpg (ctype=text/html; charset=utf-8)
[이미지 응답 이상] 200 https://www.insaartcenter.com/bbs/view_image.php?bo_table=exhibition_current&fn=f908992e10ec9c87f45e0f975130eaa9_kuhVd7lM_2bbe3cdaf60567d58870083099023088f61c380c.jpg (ctype=text/html; charset=utf-8)
[이미지 응답 이상] 200 https://www.insaartcenter.com/bbs/view_image.php?bo_table=exhibition_current&fn=f908992e10ec9c87f45e0f975130eaa9_9TWgPX2f_bf639a67a0933b56b5430a85e99a70328f5a592a.jpg (ctype=text/html; charset=utf-8)
[이미지 응답 이상] 200 https://www.insaartcenter.com/bbs/view_image.php?bo_table=exhibition_current&fn=f908992e10ec9c87f45e0f975130eaa9_aVMne20U_e731fd1a5138f2a6d1a9641f0ee4905832530073.jpg (ctype=text/html; charset=utf-8)
[이미지 응답 이상] 200 https://www.insaartcenter.com/bbs/view_image.php?bo_table=exhibition_current&fn=f908992e10ec9c87f45e0f975130eaa9_4dsWjAH

(8,
 ['전시명',
  '부주제',
  '전시장소',
  '전시기간',
  '작가',
  '전시관',
  '전시회 설명',
  'url',
  '전시 사진',
  '저장된_이미지_경로'])

In [95]:
# 저장
os.makedirs("outputs", exist_ok=True)
csv_path = "outputs/insaart_current_exhibitions.csv"
json_path = "outputs/insaart_current_exhibitions.json"

df.to_csv(csv_path, index=False, encoding="utf-8-sig")
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

csv_path, json_path

('outputs/insaart_current_exhibitions.csv',
 'outputs/insaart_current_exhibitions.json')